In [1]:
# imports
import csv
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from IPython.display import Image
import seaborn as sns

C:\Users\Arina\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Load the pickled cleaned encounters-level data
df = pd.read_pickle('../data_local/whale_encounters_clean.pkl')

In [3]:
df.head()

,additional_comments,beh_role,behavior,encounter_date,encounter_id,group_behavior,group_type,individual_id,number_calves,occurrence_id,...,submitter,url,vessel,location_name,location_id,latitude,longitude,est_group_size,sex,scarring_flag
0,,,Solo,2004-08-16 09:17:00,10059,Fluke Slap,Single,460044,0.0,DFOC:Mon Aug 16 00:00:00 EDT 2004:N2:2,...,Department of Fisheries and Oceans Canada,http://www.splashcatalog.org:80/encounters/enc...,N2,British Columbia,NBC,52.5420,-131.2021,1.0,male,0.0
1,,,Unspecified competitive group member,2004-03-29 12:53:00,1006,Competitive,Competitive Group,420485,0.0,UABCS:Mon Mar 29 00:00:00 EST 2004:RA:6,...,Universidad Autonoma de Baja California Sur,http://www.splashcatalog.org:80/encounters/enc...,RA,Baja California,MX-BC,23.3882,-109.3198,6.0,male,0.0
2,,AD,Unknown,2004-08-26 09:00:00,10056,Feeding,UNKN,472177,1.0,UAFK:Thu Aug 26 00:00:00 EDT 2004:DU:1,...,"University of Alaska, Fairbanks, Kodiak",http://www.splashcatalog.org:80/encounters/enc...,DU,Kodiak Island,NGOA,58.6260,-152.3027,40.0,male,0.0
3,,UN,Unknown,2004-08-26 09:00:00,10055,Feeding,UNKN,472380,1.0,UAFK:Thu Aug 26 00:00:00 EDT 2004:DU:1,...,"University of Alaska, Fairbanks, Kodiak",http://www.splashcatalog.org:80/encounters/enc...,DU,Kodiak Island,NGOA,58.6260,-152.3027,40.0,unknown,0.0
4,Mom?,G3,"Trio, non-competitive",2004-09-25 16:20:00,10044,,Mom calf Escort,450262,0.0,CRC:Sat Sep 25 00:00:00 EDT 2004:N2:6,...,Cascadia Research Collective,http://www.splashcatalog.org:80/encounters/enc...,N2,California,CA-OR,37.6810,-122.8631,3.0,unknown,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19185 entries, 0 to 19184
Data columns (total 23 columns):
additional_comments    19185 non-null object
beh_role               19185 non-null object
behavior               19185 non-null object
encounter_date         19185 non-null datetime64[ns]
encounter_id           19185 non-null object
group_behavior         19185 non-null object
group_type             19185 non-null object
individual_id          19185 non-null object
number_calves          13670 non-null float64
occurrence_id          19185 non-null object
photographer           19185 non-null object
region_name            19185 non-null object
sighting_comments      19185 non-null object
submitter              19185 non-null object
url                    19185 non-null object
vessel                 19185 non-null object
location_name          19185 non-null object
location_id            19185 non-null object
latitude               19185 non-null float64
longitude              191

In [6]:
df['individual_id'].value_counts()

420248    35
460293    27
420195    27
420183    25
460065    24
420270    24
420159    23
460336    22
550159    21
460388    21
420479    21
420560    20
420412    19
420389    19
420268    19
430520    19
420674    19
420144    18
540037    18
440144    18
474377    18
420200    18
420656    18
420464    18
420286    17
550364    17
574032    17
420422    17
420661    17
430107    17
          ..
574193     1
630903     1
530154     1
430212     1
572231     1
560073     1
430645     1
700235     1
430120     1
630650     1
450088     1
560016     1
620256     1
700474     1
470460     1
420575     1
510031     1
700115     1
620246     1
700268     1
572537     1
460376     1
530213     1
440171     1
450096     1
631071     1
631066     1
470949     1
474296     1
472474     1
Name: individual_id, Length: 8489, dtype: int64

In [8]:
df[df['individual_id'] == '420195']

,additional_comments,beh_role,behavior,encounter_date,encounter_id,group_behavior,group_type,individual_id,number_calves,occurrence_id,...,submitter,url,vessel,location_name,location_id,latitude,longitude,est_group_size,sex,scarring_flag
596,dorsal matches to other sightings of T447 scra...,SE,CH/SE,2005-02-26 10:25:00,15438,,CG,420195,0.0,HSU:Sat Feb 26 00:00:00 EST 2005:PA:2,...,Humboldt State University,http://www.splashcatalog.org:80/encounters/enc...,PA,Revillagigedo,MX-AR,18.7294,-110.9842,3.0,male,0.0
609,"no singers, no groups, windy, return to shore ...",MO,Mother with calf of year,2005-03-01 12:18:00,15443,,MCE,420195,1.0,HSU:Tue Mar 01 00:00:00 EST 2005:PA:4,...,Humboldt State University,http://www.splashcatalog.org:80/encounters/enc...,PA,Revillagigedo,MX-AR,18.7795,-111.0373,3.0,male,0.0
891,,CH,CH/SE,2005-02-20 15:51:00,15975,,CG,420195,0.0,UNAM:Sun Feb 20 00:00:00 EST 2005:LO:2,...,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.8608,-110.8736,3.0,male,0.0
1291,,SE,CH/SE,2005-02-19 14:38:00,15306,,CG,420195,0.0,UNAM:Sat Feb 19 00:00:00 EST 2005:LO:7,...,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.8480,-111.0245,4.0,male,0.0
1916,"B1 of morning, called G7 in notes field confus...",S/J,Pair,2005-02-23 11:03:00,14166,,SJ,420195,0.0,HSU:Wed Feb 23 00:00:00 EST 2005:PA:1,...,Humboldt State University,http://www.splashcatalog.org:80/encounters/enc...,PA,Revillagigedo,MX-AR,18.7248,-110.9737,2.0,male,0.0
1963,,CH,CH/SE,2005-02-13 12:00:00,14090,,CG,420195,0.0,UNAM:Sun Feb 13 00:00:00 EST 2005:LO:2,...,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.7116,-110.9601,3.0,male,0.0
2494,,,Pair,2005-02-21 16:46:00,13326,,2,420195,0.0,UNAM:Mon Feb 21 00:00:00 EST 2005:LO:7,...,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.8291,-111.0184,2.0,male,0.0
2552,,SE,CH/SE,2005-03-03 10:18:00,13177,,CG,420195,0.0,UNAM:Thu Mar 03 00:00:00 EST 2005:LO:1,...,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.7190,-110.9200,3.0,male,0.0
2555,,SE,Unknown,2005-03-03 16:02:00,13179,,2,420195,0.0,UNAM:Thu Mar 03 00:00:00 EST 2005:LO:6,...,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.7508,-111.0014,2.0,male,0.0
3617,,NA,Nuclear Animal in a competitive group,2004-02-19 00:00:00,1084,,Competitive Group,420195,NaN,UNAM:Thu Feb 19 00:00:00 EST 2004:LO:7,...,Universidad Nacional Autonoma de Mexico,http://www.splashcatalog.org:80/encounters/enc...,LO,Revillagigedo,MX-AR,18.7155,-110.9155,NaN,male,0.0


In [9]:
df[df['individual_id'] == '420248']['encounter_date'].dt.date.value_counts()

2005-03-29    3
2004-03-23    2
2004-03-20    2
2004-02-25    2
2006-03-23    2
2005-03-17    2
2004-03-07    2
2005-03-28    1
2006-03-07    1
2004-03-22    1
2004-03-13    1
2004-03-12    1
2005-03-31    1
2004-03-11    1
2005-04-06    1
2004-02-20    1
2005-03-05    1
2006-04-02    1
2004-03-09    1
2005-03-16    1
2005-02-23    1
2005-02-26    1
2006-02-24    1
2004-02-21    1
2004-03-26    1
2006-03-15    1
2005-03-12    1
Name: encounter_date, dtype: int64